In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import math

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
customer = pd.read_pickle("../new_data/data_set/customer.pkl")
online = pd.read_pickle("../new_data/data_set/online.pkl")
product = pd.read_pickle("../new_data/data_set/product.pkl")
transaction = pd.read_pickle("../new_data/data_set/transaction.pkl")
clac = pd.read_pickle("../new_data/data_set/offline_clac.pkl")
group = pd.read_pickle("../new_data/data_set/offline_group.pkl")

# Transaction 

In [ ]:
buy = transaction.loc[transaction['biz_unit']=='B']
buy[['buy_am','buy_ct']] = buy[['buy_am','buy_ct']].astype(float)
buy_ct = buy[['clnt_id','buy_ct']].groupby('clnt_id').sum()
buy_am = buy[['clnt_id','buy_am']].groupby('clnt_id').sum()
tran_freq = buy[['clnt_id','trans_id']]
tran_freq = tran_freq.drop_duplicates(['clnt_id','trans_id'],keep='first')
tran_freq['tran_freq'] = 1
tran_freq = tran_freq[['clnt_id','tran_freq']]
tran_freq = tran_freq.groupby('clnt_id').sum()
tran_ct = buy[['clnt_id','trans_id','buy_ct']].groupby(['clnt_id','trans_id']).sum()
tran_ct = tran_ct.groupby('clnt_id').mean()
tran_ct.columns = ['tran_ct']
tran_am = buy[['clnt_id','trans_id','buy_am']].groupby(['clnt_id','trans_id']).sum()
tran_am = tran_am.groupby('clnt_id').mean()
tran_am.columns = ['tran_am']

In [ ]:
clac_var = (clac != 0).astype(int)
clac_var = pd.DataFrame(clac_var.sum(axis=1))
clac_var.columns = ['clac_var']
clac_ct = pd.merge(buy_ct,clac_var,on='clnt_id',how='left')
clac_ct['clac_ct'] = clac_ct['buy_ct']/clac_ct['clac_var']
clac_ct = pd.DataFrame(clac_ct['clac_ct'])
clac_am = pd.merge(buy_am,clac_var,on='clnt_id',how='left')
clac_am['clac_am'] = clac_am['buy_am']/clac_am['clac_var']
clac_am = pd.DataFrame(clac_am['clac_am'])

In [ ]:
basic = pd.merge(buy_ct,buy_am,on='clnt_id',how='left')
basic = pd.merge(basic,tran_freq,on='clnt_id',how='left')
basic = pd.merge(basic,tran_ct,on='clnt_id',how='left')
basic = pd.merge(basic,tran_am,on='clnt_id',how='left')
basic = pd.merge(basic,clac_var,on='clnt_id',how='left')
basic = pd.merge(basic,clac_ct,on='clnt_id',how='left')
basic = pd.merge(basic,clac_am,on='clnt_id',how='left')
basic.to_pickle("../new_data/offline/basic.pkl")

# Time 

## 1) week 

In [ ]:
tran_time = transaction.loc[transaction['biz_unit']=='B']
tran_time['month'] = np.array([i.month for i in tran_time['date_time']])
tran_time['day'] = np.array([i.day for i in tran_time['date_time']])
tran_time['time'] = np.array([i.hour for i in tran_time['date_time']])
tran_time['weekday'] = np.array([i.weekday() for i in tran_time['date_time']])

In [ ]:
weekday = tran_time[['clnt_id','weekday']]
weekday['count'] = 1
weekday = pd.pivot_table(weekday, index='clnt_id',columns='weekday',values='count',aggfunc=sum)
weekday = weekday.fillna(0)
weekday.columns = ['mon','tue','wed','thu','fri','sat','sun']

In [ ]:
week_plot = pd.melt(weekday,var_name='days',value_name='count')
sns.boxplot(x='days',y='count',data=week_plot, showfliers=False)
plt.title('distribution of transaction by weekdays')
plt.show()

In [ ]:
week = weekday[['mon','tue','wed','thu','fri']]
week = pd.DataFrame(week.sum(axis=1))
sat = pd.DataFrame(weekday['sat'])
sun = pd.DataFrame(weekday['sun'])
weekday = pd.merge(week,sat,on='clnt_id',how='outer')
weekday = pd.merge(weekday,sun,on='clnt_id',how='outer')
weekday.columns = ['week','sat','sun']

In [ ]:
weekday['week'] = weekday['week']/5

In [ ]:
week_plot = pd.melt(weekday,var_name='days',value_name='count')
sns.boxplot(x='days',y='count',data=week_plot, showfliers=False)
plt.title('distribution of transaction by week / weekend')
plt.show()

## 2) time

In [ ]:
time = tran_time[['clnt_id','time']]
time['count'] = 1
time = pd.pivot_table(time, index='clnt_id',columns='time',values='count',aggfunc=sum)
time = time.fillna(0)

In [ ]:
time_plot = pd.melt(time,var_name='time',value_name='count')
sns.boxplot(x='time',y='count',data=time_plot.loc[time_plot['count']!=0], showfliers=False)
plt.title('distribution of transaction by time')
plt.show()

In [ ]:
tran_time['timezone'] = 0
tran_time['timezone'].loc[[i in [0,1,2,3,4,5,6,7,8,9] for i in tran_time['time']]] = 1
tran_time['timezone'].loc[[i in [10,11,12,13,14,15,16,17,18,19,20,21,22,23] for i in tran_time['time']]] = 2
# tran_time['timezone'].loc[[i in [15,16,17,18,19,20,21] for i in tran_time['time']]] = 3

In [ ]:
time = tran_time[['clnt_id','timezone']]
time['count'] = 1
time = pd.pivot_table(time, index='clnt_id',columns='timezone',values='count',aggfunc=sum)
time = time.fillna(0)
time.columns = ['time'+str(i+1) for i in range(2)]

In [ ]:
time['time1'] = time['time1']/10
time['time2'] = time['time2']/14
# time['time3'] = time['time3']/7

In [ ]:
time_plot = pd.melt(time,var_name='timezone',value_name='count')
sns.boxplot(x='timezone',y='count',data=time_plot.loc[time_plot['count']!=0], showfliers=False)
plt.title('distribution of transaction by timezone')
plt.show()

In [ ]:
time = pd.merge(weekday,time,on='clnt_id',how='outer')
time.to_pickle("../new_data/offline/time.pkl")

# Group 

In [ ]:
off_tran = transaction.loc[transaction['biz_unit']=='B']
off_tran['pd_c'] = off_tran['pd_c'].astype(int)
group['pd_c'] = group['pd_c'].astype(int)

In [ ]:
off_group = pd.merge(off_tran[['clnt_id','pd_c']],group[['pd_c','group']],on='pd_c',how='left')
off_group['count'] = 1
off_group = pd.pivot_table(off_group,index='clnt_id',columns='group',values='count',aggfunc=sum)
off_group = off_group.fillna(0)

In [ ]:
off_group.to_pickle("../new_data/offline/off_group.pkl")